In [ ]:
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd 
from mypackage import cleaning_functions
from mypackage import scraping_functions as sf
game_url = 'https://www.pro-football-reference.com/boxscores/202309070kan.htm'
df = sf.scrape_game_data(game_url)
def seconds_left(plays):
    seconds_left = abs(plays['play_start_time'] - 60) * 60
    return seconds_left
df = sf.scrape_game_data(game_url)
def adjusted_score_calc(plays):
    gamma = .5
    
    adjusted_score = plays['score_diff'] / ((plays['seconds_left']) + 1) ** gamma
    return adjusted_score



def win(plays):
        #plays = pd.DataFrame(plays),
    winning_team = None
    
        # Get the column names dynamically,
    home_team = plays.columns[6]
    vis_team = plays.columns[5]
    Y = []
    
        # Compare who is the winning team,
    if plays.iloc[-1, 5] >= plays.iloc[-1, 6]:
        winning_team = vis_team
    else:
            winning_team = home_team
            
    
        # Figure out who is winning at the end,
    for _, play in plays.iterrows():
        if play['possession'] == winning_team:
                Y.append(1),
        else:
            Y.append(0)
    return Y
df['seconds_left'] = df.apply(seconds_left,axis =1),
df['score_diff'] = df.iloc[:,5].astype(float) - df.iloc[:,6].astype(float)
df['adjusted_score'] = df.apply(adjusted_score_calc,axis=1)
df['win_team'] = df.apply(lambda row: win(pd.DataFrame(row).transpose()), axis=1)
    # Define the column indices to drop,
columns_to_drop = [5, 6]
    
    # Drop the specified columns,
new_data = df.drop(columns=df.columns[columns_to_drop])
    
    # Optionally, you can also drop additional columns,
additional_columns_to_drop = ['Quarter', 'Detail', 'Location', 'Time', 'win_team', 'play_start_time']
new_data = new_data.drop(columns=additional_columns_to_drop)
new_data['field_side'] = pd.Categorical(new_data['field_side'])
new_data['Play_Type'] = pd.Categorical(new_data['Play_Type'])
new_data['possession'] = pd.Categorical(new_data['possession'])
new_data['Down'] = pd.Categorical(new_data['Down'])
new_data['ToGo'] = pd.to_numeric(new_data['ToGo'], errors='coerce')
new_data['ToGo'] = new_data['ToGo'].astype(float)
new_data['EPB'] = new_data['EPB'].astype(float)
new_data['EPA'] = new_data['EPA'].astype(float)
new_data = pd.get_dummies(new_data, columns=['Play_Type', 'Down', 'field_side', 'possession']),
y = df['win_team'],
y = y.apply(lambda x:float( x[0]))
new_data = new_data.fillna(0)
X_train, X_test, y_train, y_test = train_test_split(new_data, y, test_size=.2, random_state=42, stratify=y)
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_classifier.fit(X_train, y_train)


y_pred_prob = rf_classifier.predict_proba(X_test)
    #accuracy = accuracy_score(y_test, y_pred),
    #accuracy,
y_pred_labels = np.argmax(y_pred_prob, axis=1)

classification_rep = classification_report(y_test, y_pred_labels)



X_combined_proba = pd.concat([X_test, pd.DataFrame(y_pred_prob, columns=['Prob_0', 'Prob_1'])], axis=1)
X_combined_proba = pd.DataFrame()
X_combined_proba.columns

def plot_decision_boundary(model, variable, values, title=None):
        plt.figure(figsize=(10, 6))
        for value in values:
            new_data[variable] = value
            X_new = new_data[['seconds_left']].values
            #y_pred_prob = model.predict_proba(X_new)[:,1],
            sns.scatterplot(x=new_data[variable][:len(y_pred_prob)].values, y=y_pred_prob[:,1], label=f'{variable}={value}')
    
        plt.title(title),
        plt.xlabel(variable),
        plt.ylabel('Probability of Class 1')
        plt.legend()
        plt.show()



plot_decision_boundary(rf_classifier, 'seconds_left', [3600,3150, 2700, 2250, 1800, 1350, 900, 450, 0], title='Win Probability (Random Forest)')
X_train, X_test, y_train, y_test = train_test_split(new_data, y, test_size=.2, random_state=42, stratify=y)
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_classifier.fit(X_train, y_train)


y_pred_prob = rf_classifier.predict_proba(X_test)
    #accuracy = accuracy_score(y_test, y_pred),
    #accuracy,
y_pred_labels = np.argmax(y_pred_prob, axis=1)

classification_rep = classification_report(y_test, y_pred_labels)



X_combined_proba = pd.concat([X_test, pd.DataFrame(y_pred_prob, columns=['Prob_0', 'Prob_1'])], axis=1)
X_combined_proba = pd.DataFrame()
X_combined_proba.columns

def plot_decision_boundary(model, variable, values, title=None):
        plt.figure(figsize=(10, 6))
        for value in values:
            new_data[variable] = value
            X_new = new_data[['seconds_left']].values
            #y_pred_prob = model.predict_proba(X_new)[:,1],
            sns.scatterplot(x=new_data[variable][:len(y_pred_prob)].values, y=y_pred_prob[:,1], label=f'{variable}={value}')
    
        plt.title(title)
        plt.xlabel(variable)
        plt.ylabel('Probability of Class 1')
        plt.legend()
        plt.show()



plot_decision_boundary(rf_classifier, 'seconds_left', [3600,3150, 2700, 2250, 1800, 1350, 900, 450, 0], title='Win Probability (Random Forest)')
X_play_by_play = new_data
    
    # Predict win probabilities using the trained RandomForestClassifier,
predicted_win_probabilities_rf = rf_classifier.predict_proba(X_play_by_play)[:, 1]
    
    # Create a new DataFrame combining play-by-play data and predicted win probabilities,
play_by_play_with_prob_rf = new_data.reset_index()
play_by_play_with_prob_rf['predicted_win_probability'] = predicted_win_probabilities_rf
    
    # Plotting,
plt.figure(figsize=(12, 6))
sns.lineplot(x='index', y='predicted_win_probability', data=play_by_play_with_prob_rf)
plt.xlabel('Play Number')
plt.ylabel('Predicted Win Probability')
plt.title('Predicted Win Probability Over Plays')
plt.show()
log_model = LogisticRegression(solver='liblinear')
log_model.fit(X_train, y_train)
 
Log_y_pred = log_model.predict_proba(X_test)
log_y_labels = np.argmax(Log_y_pred, axis=1)
accuracy = accuracy_score(y_test, log_y_labels)
conf_matrix = confusion_matrix(y_test, log_y_labels)
log_classification_rep = classification_report(y_test,log_y_labels)

X_play_by_play = new_data
    
    # Predict win probabilities using the trained RandomForestClassifier,
predicted_win_probabilities = log_model.predict_proba(X_play_by_play)[:, 1],
    
    # Create a new DataFrame combining play-by-play data and predicted win probabilities,
play_by_play_with_prob = new_data.reset_index(),
play_by_play_with_prob['predicted_win_probability'] = predicted_win_probabilities
plt.figure(figsize=(12, 6))
sns.lineplot(x='index', y='predicted_win_probability', data=play_by_play_with_prob),
plt.xlabel('Play Number'),
plt.ylabel('Predicted Win Probability'),
plt.title('Predicted Win Probability Over Plays'),
plt.show()

